Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [1]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-11-06 01:47:29--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.185.29
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.185.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  55.0MB/s    in 3.6s    

2019-11-06 01:47:32 (55.0 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [2]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [3]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [4]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [0]:
fruits = ["Banana", 'Bag of Organic Bananas', 'Organic Strawberries', 
          'Organic Baby Spinach', 'Organic Hass Avacado', 'Organic Avacado', 
          'Large Lemon', 'Strawberries', 'Limes', 'Organic Whole Milk']

In [0]:
#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns

In [7]:
# Reading in each of the data sets to see what the columns are
order_products__prior = pd.read_csv('order_products__prior.csv')
order_products__prior.head(3)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0


In [9]:
order_products__train = pd.read_csv('order_products__train.csv')
order_products__train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [10]:
orders = pd.read_csv('orders.csv')
orders.head(3)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0


In [40]:
products = pd.read_csv('products.csv')

products.head(3)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7


In [51]:
#seeing the value_counts of the products

products['product_name'].value_counts(ascending =False)

Lubricant /Redness Reliever Eye Drops, Maximum Redness Relief    1
Shirataki Zero Calorie Noodle Substitute                         1
Sensitive Skin Shave Gel with Aloe                               1
Organic Popped Corn Dill Pickle                                  1
Red Plastic Cups                                                 1
                                                                ..
Ultra Thin Slices Swiss Cheese                                   1
Margherite Combination Cookies                                   1
Snug & Dry Diapers Step 1 Jumbo                                  1
Sweet Red Pepper Hummus                                          1
Rice & Vegetables Stir Fry                                       1
Name: product_name, Length: 49688, dtype: int64

In [12]:
aisles = pd.read_csv('aisles.csv')
aisles.head(3)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars


In [13]:
departments = pd.read_csv('departments.csv')
departments.head(3)

,department_id,department
0,1,frozen
1,2,other
2,3,bakery


In [17]:
# it appears that I will want to combine the train and the prior order datasets 
# and then merge with the products dataset after filtering out some
# of the columns

#will concat the two order_products train and prior
order_products = pd.concat([order_products__prior, order_products__train])
print(order_products.shape)
order_products.sample(15)


(33819106, 4)


,order_id,product_id,add_to_cart_order,reordered
12394094,1308235,32655,8,1
8577174,905465,21137,1,1
21397175,2256955,40592,3,1
14449170,1524661,6184,4,1
12933857,1365237,47766,4,1
20259167,2136848,44142,17,0
466464,1143333,31334,11,1
4853533,512290,24852,2,1
8810219,930019,27092,2,1
10577649,1116649,17281,4,1


In [38]:
#slimming down the order_products dataframe to just the order id
# which is a unique order and the product id
order_products_subset = order_products[['order_id', 'product_id']]
print(order_products_subset.shape)
order_products_subset.head()

(33819106, 2)


,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035


In [20]:
#will be filtering the products dataframe
products_subset = products[['product_id', 'product_name']]
print(products_subset.shape)
products_subset.sample(3)

(49688, 2)


,product_id,product_name
37068,37069,Liquid Drain Cleaner
1834,1835,Caffeine Free Diet
30827,30828,Burger Veggie Curry


In [0]:
# the list of the fruits that are most common
fruits = ["Banana", 'Bag of Organic Bananas', 'Organic Strawberries', 
          'Organic Baby Spinach', 'Organic Hass Avacado', 'Organic Avacado', 
          'Large Lemon', 'Strawberries', 'Limes', 'Organic Whole Milk']

In [0]:
def fruitInCol(theDF):
  fruits = ["Banana", 'Bag of Organic Bananas', 'Organic Strawberries', 
          'Organic Baby Spinach', 'Organic Hass Avacado', 'Organic Avacado', 
          'Large Lemon', 'Strawberries', 'Limes', 'Organic Whole Milk']
  newList = []
  for i in theDF['product_name']:
    if i in set(fruits):
      newList.append(True)
    else: 
      newList.append(False)
  return newList

In [0]:
#filtering by rows on the, creating the condition
condition = fruitInCol(products_subset)


In [36]:
#filtering the product_subset
product_subset_subset = products_subset[condition]
print(product_subset_subset.shape)
product_subset_subset

(8, 2)


,product_id,product_name
13175,13176,Bag of Organic Bananas
16796,16797,Strawberries
21136,21137,Organic Strawberries
21902,21903,Organic Baby Spinach
24851,24852,Banana
26208,26209,Limes
27844,27845,Organic Whole Milk
47625,47626,Large Lemon


In [46]:
#will try to merge the dataframe of product_subset and order_products_subset
merged = pd.merge(products_subset, order_products_subset, on='product_id',
                  how='inner')
print(merged.shape)
merged.sample(5)

(33819106, 3)


,product_id,product_name,order_id
18083492,26940,Organic Large Green Asparagus,1321584
1107655,2091,Almond Nut & Rice Cracker Snacks,2781068
325877,436,Petite Brussels Sprouts,907914
15278955,23417,Brown Rice & Vegetables Bowl,1070410
6325941,10385,Organic English Cucumber,507945


In [67]:
pd.options.display.max_rows= 99
merged['product_name'].value_counts(ascending=False).head(10)

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code
- Complete the code cells we skipped near the beginning of the notebook
- Table 2 --> Tidy
- Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [62]:
#creating the dataSet for table1 will then transpose for table2
table1 = pd.DataFrame({'treatmenta': [np.nan, 16, 3], 'treatmentb': [2, 11, 1]},
                      index=["John Smith", "Jane Doe", "Mary Johnson"])
table1

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


In [63]:
table2 = table1.T
table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [68]:
#moving the index to become a column
table2 = table2.reset_index()
table2

,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [69]:
#changing to tidy format using the melt method
tidy_table2 = table2.melt(id_vars='index', value_vars=["John Smith", "Jane Doe",
                                                       'Mary Johnson'])
tidy_table2


,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [70]:
#will now go back to table 2 from the tidy_table2
wide_table2 = tidy_table2.pivot_table(index='index'   , columns='variable'  
                                      , values='value')
wide_table2

variable,Jane Doe,John Smith,Mary Johnson
index,,,
treatmenta,16.0,NaN,3.0
treatmentb,11.0,2.0,1.0


In [71]:
wide_table2.columns

Index(['Jane Doe', 'John Smith', 'Mary Johnson'], dtype='object', name='variable')

In [72]:
wide_table2.index

Index(['treatmenta', 'treatmentb'], dtype='object', name='index')

In [73]:
wide_table2

variable,Jane Doe,John Smith,Mary Johnson
index,,,
treatmenta,16.0,NaN,3.0
treatmentb,11.0,2.0,1.0


In [0]:
flights = sns.load_dataset('flights')

In [58]:
#finding the colums of the flights dataframe
print(flights.shape)
flights.sample(3)


(144, 3)


,year,month,passengers
137,1960,June,535
83,1955,December,278
136,1960,May,472


In [59]:
#showing the columns of the datafram
flights.columns

Index(['year', 'month', 'passengers'], dtype='object')

In [74]:
#creating a pivot table
flights_pivot = flights.pivot_table(index='year', columns='month' )
flights_pivot

passengers                       ...                                    
month    January February March April  ... September October November December
year                                   ...                                    
1949         112      118   132   129  ...       136     119      104      118
1950         115      126   141   135  ...       158     133      114      140
1951         145      150   178   163  ...       184     162      146      166
1952         171      180   193   181  ...       209     191      172      194
1953         196      196   236   235  ...       237     211      180      201
1954         204      188   235   227  ...       259     229      203      229
1955         242      233   267   269  ...       312     274      237      278
1956         284      277   317   313  ...       355     306      271      306
1957         315      301   356   348  ...       404     347      305      336
1958         340      318   362   348  ...       404     359      310      337
1959         360      342   406   396  ...       463     407      362      405
1960         417      391   419   461  ...       508     461      390      432

[12 rows x 12 columns]

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####